In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from  tensorflow.keras.preprocessing.sequence import pad_sequences

2023-11-30 01:29:00.545091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
restaurant_data = pd.read_csv("final_df.csv")

In [3]:
restaurant_data.columns

Index(['name', 'cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale',
       'review_objects', 'dishes', 'price', 'type', 'popular_times',
       'in_cuisine', 'japanese', 'chinese', 'bar', 'western', 'indian',
       'mediterranean', 'mexican', 'italian', 'local', 'malaysian', 'hipster',
       'veg', 'fast_food', 'thai', 'korean', 'halal', 'dessert', 'seafood',
       'health', 'french', 'vietnamese', 'michellin'],
      dtype='object')

In [4]:
restaurant_data_clean  = restaurant_data.drop(['cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale', 'review_objects', 'dishes', 'type', 'popular_times', 'in_cuisine'], axis = 1)

In [5]:
restaurant_data_clean['price'].unique()

array(['$$', '$', '$$$', '$$ - $$$', '$$$$', '$$$$$'], dtype=object)

In [6]:
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$', 'price'] = 0
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$', 'price'] = 1
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$ - $$$', 'price'] = 2
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$', 'price'] = 3
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$', 'price'] = 4
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$$', 'price'] = 5

In [7]:
restaurant_name_list = restaurant_data_clean['name']
restaurant_data_clean.drop(['name'], axis = 1, inplace = True)

In [8]:
restaurant_data_clean
## Create df from streamlit

,price,japanese,chinese,bar,western,indian,mediterranean,mexican,italian,local,...,fast_food,thai,korean,halal,dessert,seafood,health,french,vietnamese,michellin
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
2,3,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4199,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4200,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4201,3,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4202,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
restaurant_data_clean.to_csv('restaurant_data_clean.csv', index = False)

In [10]:
restaurant_numpy = restaurant_data_clean.drop('price', axis = 1).to_numpy()

In [11]:
# Save restaurant_numpy
np.save('restaurant_numpy.npy', restaurant_numpy)

In [342]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Creaate a test set
        self.test_set = np.zeros(self.R.shape)
        flag = False
        i_test, j_test = 0, 0
        for i in range(self.R.shape[0]):
            for j in range(self.R.shape[1]):
                if self.R[i,j] != 0:
                    self.test_set[i,j] = self.R[i,j]
                    i_test, j_test = i, j
                    flag = True
                    break
            if flag:
                break


        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if (self.R[i, j] != 0) & ((i != i_test) | (j != j_test))
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        test_mse = self.test_mse(self.test_set)
        print("Test MSE: %.4f" % test_mse)

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return error/len(xs)
    
    def test_mse(self, test_data):
        """
        A function to compute the total mean square error
        """
        xs, ys = test_data.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(test_data[x, y] - predicted[x, y], 2)
        return error/len(xs)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Compute the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [13]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [14]:
print(restaurant_numpy, restaurant_numpy.shape)    

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]] (4204, 22)


In [15]:
restaurant_data_clean['price'].unique()

array([1, 0, 3, 2, 4, 5], dtype=object)

In [16]:
# Implement Content Based Filtering 
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
'''
# Price, Cuisine Not needed anymore
item = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])
user = np.array([[0, 1, 1], [1, 1, 0]])
'''

'\n# Price, Cuisine Not needed anymore\nitem = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])\nuser = np.array([[0, 1, 1], [1, 1, 0]])\n'

In [75]:
# Item
item = restaurant_numpy
# User
rng = np.random.default_rng(12345)


In [78]:
from scipy import stats
from scipy.sparse import random 

# Create a sparse matrix
user = rng.exponential(scale=0.5, size=(3,22))

print(user, user.shape)

[[2.77762308e-01 7.96385330e-02 8.45913354e-02 5.87402774e-01
  1.45587953e-01 7.15745135e-01 2.93521959e-01 1.54489089e-01
  2.23483962e+00 5.00034997e-02 9.03563276e-02 4.14144538e-01
  4.86540370e-01 5.22037003e-01 1.32899306e-01 1.90612347e-01
  3.94952984e-01 8.79256356e-02 1.41993431e-01 1.94433926e-01
  1.05342363e-01 4.77625228e-01]
 [3.02987285e-01 1.12840254e-02 1.78065443e-01 7.54185247e-01
  1.43283938e+00 2.12193461e-01 2.97717790e-01 1.11752735e-01
  1.87106236e+00 1.54908868e-01 6.63230256e-02 1.11108987e-01
  3.05569701e-01 2.41387647e-01 3.19399478e-01 3.63744546e-01
  9.50787740e-01 6.45487234e-01 2.97715928e-01 1.38811685e-01
  1.12599122e-01 5.43637303e-02]
 [8.15719058e-01 1.03398782e+00 5.15713403e-01 1.59408124e-02
  9.34426922e-01 1.13383573e-01 6.43501311e-01 7.54377407e-01
  4.24382827e-01 3.76634832e-01 1.25791181e-01 3.23613464e-01
  5.25635572e-01 6.17462378e-04 3.58328517e-01 1.82339611e-01
  1.20396193e-01 3.43570627e-01 1.41532547e+00 9.36851195e-01
  3.

In [79]:
# Do the above with list comprehension
user = np.array([[1 if val >= 1 else 0 for val in user] for user in user])
print(user, user.shape)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]] (3, 22)


In [80]:
# Save user array
np.save('user.npy', user)

In [88]:
def get_item_user(user_arr, items):
    user_item = []
    for user in user_arr:
        for item in items:
            cos_sim = dot(user, item.T)/(norm(user)*norm(item))
            user_item.append(cos_sim)
    user_item = np.array(user_item)
    user_item = user_item.reshape(3, 4204)

    return user_item

In [89]:
item_user = get_item_user(user, item)
print(len(item_user), len(item_user[0]))
print(["{0:0.2f}".format(i) for i in item_user[0]])

3 4204
['0.00', '0.00', '0.00', '0.58', '1.00', '0.00', '0.00', '0.00', '0.58', '0.71', '0.71', '0.71', '0.00', '0.00', '0.71', '0.71', '0.71', '0.71', '0.71', '0.58', '1.00', '0.00', '1.00', '0.71', '0.00', '1.00', '0.58', '0.71', '0.71', '0.58', '0.00', '1.00', '0.00', '0.00', '0.00', '1.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.58', '1.00', '0.71', '0.71', '0.00', '0.00', '0.00', '0.00', '0.58', '0.71', '0.00', '0.58', '0.71', '0.58', '0.71', '1.00', '0.00', '1.00', '0.58', '0.00', '0.00', '0.00', '0.00', '0.00', '0.58', '0.00', '0.00', '0.00', '0.58', '0.00', '0.00', '0.71', '0.71', '1.00', '0.00', '0.71', '1.00', '0.00', '0.00', '0.00', '0.00', '0.58', '0.00', '0.71', '1.00', '1.00', '0.58', '0.00', '0.71', '0.71', '0.00', '0.00', '0.58', '0.00', '1.00', '0.58', '0.00', '1.00', '0.71', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.71', '0.00', '0.00', '1.00', '0.71', '0.00', '0.71', '1.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 

In [330]:
print(item_user[0])

[0.         0.         0.         ... 0.70710678 0.         0.70710678]


In [91]:
print(len(item_user[0]))

4204


In [128]:
index_of_item_user = sorted(range(len(item_user[0])), key=lambda i: item_user[0][i])[-3:]

# Get the indexs of the item_users with value 1
index_of_item_user = np.where(item_user[1] >= 0.9)

In [129]:
item_user[1][index_of_item_user]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [139]:
print(user[1])
# Get the indexs of the user with value 1
index_of_cusine = np.where(user[1] == 1)
index_of_cusine = index_of_cusine[0]
print(index_of_cusine)

[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 8]


In [141]:
for index in index_of_item_user:
    print(restaurant_data_clean.iloc[index, index_of_cusine+1])

      indian  local
16         1      1
23         1      1
152        1      1
153        1      1
154        1      1
...      ...    ...
3544       1      1
3661       1      1
4064       1      1
4135       1      1
4146       1      1

[61 rows x 2 columns]


In [145]:
for i in range(1, 23):
    print(restaurant_data_clean.columns[i], user[1][i-1])

japanese 0
chinese 0
bar 0
western 0
indian 1
mediterranean 0
mexican 0
italian 0
local 1
malaysian 0
hipster 0
veg 0
fast_food 0
thai 0
korean 0
halal 0
dessert 0
seafood 0
health 0
french 0
vietnamese 0
michellin 0


In [146]:
def item_item(item):
    item_item = []
    for i in range(0, len(item)):
        item_similarity = []
        for j in range(0, len(item)):
            #if(i!=j):
            item_i = item[i]
            item_j = item[j]
            cos_sim = dot(item_i, item_j)/(norm(item_i)*norm(item_j))
            item_similarity.append(cos_sim)
        item_item.append(item_similarity)

    return item_item

item_item_matrix = item_item(item)
    
# Send as csv

In [147]:
# Save item_item_matrix
np.save('item_item_matrix.npy', item_item_matrix)

In [148]:
print(sorted(item_item_matrix[0], reverse = True))

[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475,

In [149]:
print(np.shape(item_item_matrix))

(4204, 4204)


In [239]:
# Collaborative 

#print(len(restaurant_data_clean))

user_swiped = [[-1 , 0, 1, 1], [1 ,0, 0, 1]]
user_swiped = rng.integers(low = -1, high = 2, size = (3, len(restaurant_data_clean)))
print(user_swiped.shape)
R = user_swiped
print(R.shape)
mf = MF(R, K=4, alpha=0.1, beta=0.01, iterations=100)

(3, 4204)
(3, 4204)


In [338]:
user_swiped_normal = rng.normal(0, size=(3,len(restaurant_data_clean)))
print(user_swiped_normal[0])
user_swiped_normal = np.array([[1 if val >= 2 else -1 if val<=-2 else 0 for val in user_swiped_normal] for user_swiped_normal in user_swiped_normal])
# count unique in user_swiped_normal
print(np.unique(user_swiped_normal[0], return_counts=True))

[-0.00538775 -0.76946314 -0.90911511 ... -1.07398346  1.14613467
  0.32079028]
(array([-1,  0,  1]), array([ 107, 4013,   84]))


In [343]:
R_normal = user_swiped_normal
print(R_normal.shape)
mf_normal = MF(R_normal, K=4, alpha=0.1, beta=0.01, iterations=50)
#mf = MF(R, K=4, alpha=0.1, beta=0.01, iterations=50)
training_process = mf_normal.train()
print()
print("P x Q:")
print(mf_normal.full_matrix())
print()
print("Global bias:")
print(mf_normal.b)
print()
print("User bias:")
print(mf_normal.b_u)
print()
print("Item bias:")
print(mf_normal.b_i)

(3, 4204)
Iteration: 10 ; error = 0.0047
Iteration: 20 ; error = 0.0027
Iteration: 30 ; error = 0.0027
Iteration: 40 ; error = 0.0026
Iteration: 50 ; error = 0.0024
Test MSE: 1.1192

P x Q:
[[ 0.10419891  0.07049384  0.34614366 ... -0.47302661 -0.39696469
  -0.5239338 ]
 [-0.46347006  0.01577864 -0.35302003 ...  0.79203328 -0.42339062
   0.18480375]
 [ 0.04328166  0.03391174 -0.1498101  ... -0.24333019  0.47105037
   0.33818899]]

Global bias:
-0.007194244604316547

User bias:
[-0.10856018  0.08560528  0.00316382]

Item bias:
[0. 0. 0. ... 0. 0. 0.]


In [334]:
mf.get_rating(0, 0)

0.9906721950111878

In [240]:
print(list(user_swiped[0]))

# Get unique values count in user_swiped
unique, counts = np.unique(user_swiped[0], return_counts=True)
print(dict(zip(unique, counts)))

[1, -1, 0, 1, 1, 1, 0, -1, 0, 0, -1, 0, 0, -1, 0, -1, 1, -1, -1, 0, -1, -1, -1, -1, 1, -1, 1, 1, 0, 0, 0, -1, 1, 0, 1, 0, -1, 0, 1, 0, -1, 1, -1, 0, 0, 0, 1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 0, 1, -1, 0, 0, 1, 0, 1, 0, 1, 1, -1, 0, 0, 0, -1, 1, -1, 1, 1, 0, 0, 1, 0, 1, -1, -1, 1, 0, 1, 0, 0, -1, 0, 1, 1, 1, 1, 0, -1, 1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, 1, 0, 0, -1, 1, 1, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 1, 0, -1, 0, 0, -1, 1, 0, 1, 1, 1, -1, 0, 0, 1, 0, 0, -1, 1, 1, 0, 1, 0, -1, 1, 1, -1, 0, -1, 1, -1, -1, 1, 0, 1, 0, 0, 1, 1, -1, 0, 0, -1, -1, -1, -1, 1, -1, 0, -1, -1, 1, 0, 1, 1, 1, 0, 1, 1, -1, -1, 0, 0, 1, -1, 0, 1, 1, 1, -1, -1, -1, -1, 1, 1, 0, 0, 0, 1, -1, -1, 1, -1, -1, -1, -1, 0, -1, 0, 0, 0, 0, 1, 0, 0, -1, -1, 0, -1, 1, -1, -1, 1, 0, 0, 0, 1, 0, -1, -1, -1, -1, 0, -1, -1, 1, -1, -1, 1, 1, 1, 1, 0, -1, 0, 1, 0, 1, 0, -1, 0, 0, -1, 0, 1, 0, 1, 1, -1, -1, 0, -1, 1, 1, 0, 1, 1, 0, 1, 0, 1, -1, 1, -1, -1, 1, -1, 0, 1, -1, 1, 1, 0, 0, 0, -1, -1, -1, 0, -1, -

In [344]:
mf = MF(R, K=4, alpha=0.1, beta=0.01, iterations=50)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)

Iteration: 10 ; error = 0.0023
Iteration: 20 ; error = 0.0005
Iteration: 30 ; error = 0.0005
Iteration: 40 ; error = 0.0003
Iteration: 50 ; error = 0.0003
Test MSE: 0.1681

P x Q:
[[ 0.58993982 -0.99076827 -0.2959329  ... -0.96583783  0.41161427
   0.99540508]
 [ 0.99190061 -0.57857317 -0.96867482 ...  0.99064575  0.24095957
   0.35854944]
 [ 0.99557978 -0.99668191  0.9868066  ...  0.98192086  0.99379756
   0.99638284]]

Global bias:
0.001062950277548128

User bias:
[ 0.00451307 -0.00733998  0.0159778 ]

Item bias:
[ 0.87630285 -0.86388841 -0.11020802 ...  0.32737016  0.52898936
  0.76571798]


In [248]:
"""
user_user: [user_1: [item_1, item_2, item_3, item_4], user_2: [item1, item_2, item_3, item_4]
"""
user_user = mf.full_matrix()


In [345]:
user_user_normal = mf_normal.full_matrix()

In [249]:
'''
item_item: [item1: [item_1, item_2, item_3, item_4], item_2: [item_1, item_2, item_3, item_4] ...]
'''
item_item_matrix[0]

[0.9999999999999998,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8164965809277259,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.4999999999999999,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4999999999999999,
 0.4999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,


In [250]:
'''
item_user: [user_1: [item_1, item_2, item_3, item_4], user_2:[ item_1, item_2, item_3, item_4]]
'''
item_user[0]

array([0.        , 0.        , 0.        , ..., 0.70710678, 0.        ,
       0.70710678])

In [251]:
for i in range(0, len(item_user[0])):
    if(item_user[0][i] == 0.5):
        print(item_user[0][i], user_user[0][i])

0.5 0.9812138125835287
0.5 0.9874220939703848
0.5 0.9855467128310964
0.5 0.9774632665531303
0.5 0.9703028701555205
0.5 0.9791098553045824
0.5 -0.006126302599376743
0.5 0.971905706754387
0.5 -0.7497068877365474
0.5 -0.9911545706807636
0.5 -1.0054899800416086
0.5 0.9867319314387073
0.5 -0.9861406383660825
0.5 -0.9986990706257797
0.5 -0.508956782202649
0.5 0.13790498838200607
0.5 -0.9877381266937938
0.5 0.9879660079045196
0.5 0.9834896955516924
0.5 -0.996684861199002
0.5 0.9819328652626056
0.5 0.9815879557268217
0.5 0.9824067284826196
0.5 -0.9951932260833214
0.5 0.10813338340246897
0.5 0.2621268741144705
0.5 -0.990823189268713
0.5 0.10746348624376079
0.5 -0.9964583612026826
0.5 0.9908295351908525
0.5 0.4242236946407615
0.5 -0.05321420011884847
0.5 0.7137485371402525
0.5 -0.5854021128176415
0.5 -0.9915291680099554
0.5 -0.9835398818412439
0.5 -0.9904088387237416
0.5 -0.992082096877585
0.5 0.0094493712652278
0.5 0.15714379166972514
0.5 0.8128476144890298
0.5 -0.8156387665985982
0.5 0.9688610

In [272]:
def get_collab_content(item_user_arr, user_user_arr):
    collab_content = np.array(item_user_arr) + np.array(user_user_arr) 
    return collab_content

In [346]:
collab_content_normal = get_collab_content(item_user, user_user_normal)

In [274]:
collab_content = get_collab_content(item_user, user_user).tolist()
collab_content[0]
sorted(collab_content[0], reverse= True)

[2.082809167732732,
 1.9901433295031192,
 1.989362204928463,
 1.989217001998317,
 1.9890542352671796,
 1.9885846438777137,
 1.9883024597053822,
 1.9878357704055931,
 1.987644302185962,
 1.9875779529986808,
 1.9874793755999784,
 1.987468809374771,
 1.987284903377304,
 1.9872097300258003,
 1.987034405762412,
 1.986989330703522,
 1.9869387536156422,
 1.9868699754337227,
 1.986469662109815,
 1.9862417470177625,
 1.9859809664023413,
 1.9858824656958476,
 1.9854278296759627,
 1.9851197782774,
 1.985038322335246,
 1.985005766269441,
 1.9848937469891785,
 1.9845530766923294,
 1.9845116351490706,
 1.9844635637068606,
 1.9844032883334637,
 1.9843827304453525,
 1.983864891496383,
 1.9836868169438624,
 1.9836694972846756,
 1.9836011040920258,
 1.9835847105819466,
 1.9832739532678496,
 1.98275761696316,
 1.982702172847341,
 1.9824053818855982,
 1.9822069768109705,
 1.9820990143244563,
 1.98186927807396,
 1.9812078776024653,
 1.981100737335437,
 1.981006043017373,
 1.980956828208712,
 1.980861581436

In [254]:
def get_recommendation(user_index, item_item_matrix, collab_content):
    user_arr = collab_content[user_index]
    recommendation = user_arr.index(max(user_arr))
    print(recommendation)
    # Recommend the top 5 which the user has not swiped on
    n_recommendation = sorted(range(len(item_item_matrix[recommendation])), key=lambda i: item_item_matrix[recommendation][i], reverse = True)
    for i in n_recommendation:
        if(user_swiped[user_index][i] == 1 or user_swiped[user_index][i] == -1):
            n_recommendation.remove(i)
    #print(item_item_matrix[recommendation])
    #print(sorted(item_item_matrix[recommendation], reverse=True))
    print(n_recommendation)
    return n_recommendation

In [275]:
def get_recommendation(user_index, item_item_matrix, collab_content):
    user_arr = collab_content[user_index]
    recommendation = user_arr.index(max(user_arr))
    print(recommendation)
    # Recommend the top 5 which the user has not swiped on
    n_recommendation = sorted(range(len(item_item_matrix[recommendation])), key=lambda i: item_item_matrix[recommendation][i], reverse = True)[:5]
    #print(item_item_matrix[recommendation])
    #print(sorted(item_item_matrix[recommendation], reverse=True))
    print(n_recommendation)
    return n_recommendation

In [371]:
def get_recommendation(user_index, collab_content):
    user_arr = collab_content[user_index]
    # Get the index of the top 5 values in user_arr
    n_recommendation = sorted(range(len(user_arr)), key=lambda i: user_arr[i], reverse = True)[:5]
    #recommendation = user_arr.index(max(user_arr))
    #print(recommendation)
    # Recommend the top 5 which the user has not swiped on
    # n_recommendation = sorted(range(len(item_item_matrix[recommendation])), key=lambda i: item_item_matrix[recommendation][i], reverse = True)[:5]
    #print(item_item_matrix[recommendation])
    #print(sorted(item_item_matrix[recommendation], reverse=True))
    
    print(n_recommendation)
    return n_recommendation

In [367]:
def get_recommendation(user_index, collab_content):
    user_arr = collab_content[user_index]
    # Get the index of the top 5 values in user_arr
    n_recommendation = sorted(range(len(user_arr)), key=lambda i: user_arr[i], reverse = True)
    n_recommendation_short = []
    # Cut recommandations which have been swiped on
    
    for i in n_recommendation:
        if(user_swiped[user_index][i] == 1 or user_swiped[user_index][i] == -1):
            continue
        else:
            n_recommendation_short.append(i)
    print(n_recommendation_short)
    return n_recommendation_short

In [277]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro')
    recall = recall_score(y_true, y_pred, average = 'micro')
    f1 = f1_score(y_true, y_pred, average = 'micro')
    return accuracy, precision, recall, f1

# Get the metrics for the user
def get_user_metrics(user_index, user_swiped, n_recommendation):
    y_true = user_swiped[user_index][n_recommendation]
    y_pred = np.zeros(len(y_true))
    for i in range(0, len(n_recommendation)):
        y_pred[i] = 1
    accuracy, precision, recall, f1 = get_metrics(y_true, y_pred)
    return accuracy, precision, recall, f1

get_user_metrics(0, user_swiped, get_recommendation(0, item_item_matrix, collab_content))


1161
[4, 20, 22, 25, 31]


(0.2, 0.2, 0.2, 0.20000000000000004)

In [281]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro')
    recall = recall_score(y_true, y_pred, average = 'micro')
    f1 = f1_score(y_true, y_pred, average = 'micro')
    return accuracy, precision, recall, f1

# Get the metrics for the user
def get_user_metrics(user_index, user_swiped, n_recommendation):
    y_true = user_swiped[user_index][n_recommendation][0:5]
    y_pred = np.zeros(len(y_true))
    for i in range(0, 5):
        y_pred[i] = 1
    accuracy, precision, recall, f1 = get_metrics(y_true, y_pred)
    return accuracy, precision, recall, f1

get_user_metrics(0, user_swiped, get_recommendation(0, collab_content))


[1161, 2831, 937, 1888, 481]


(0.8, 0.8, 0.8, 0.8000000000000002)

In [283]:
print(collab_content[0][1161])

2.082809167732732


In [282]:
print(user_swiped[0][get_recommendation(0, collab_content)])

[1161, 2831, 937, 1888, 481]
[0 1 1 1 1]


In [279]:
print(user_swiped[0][20])

-1


In [368]:
recco = get_recommendation(1, collab_content)

[228, 3310, 3041, 1899, 347, 3661, 16, 1687, 1646, 3564, 230, 501, 266, 252, 2872, 2841, 737, 156, 1639, 1468, 3071, 74, 3008, 890, 248, 321, 777, 1486, 1601, 1341, 309, 297, 78, 486, 4170, 2475, 2404, 4101, 937, 2902, 527, 3930, 1387, 2006, 635, 3103, 2479, 522, 222, 2618, 3538, 2281, 1360, 1880, 2513, 4120, 3300, 3817, 2048, 2530, 481, 2406, 745, 3818, 3027, 1368, 2036, 2764, 2287, 2831, 4064, 721, 876, 2077, 2896, 734, 1714, 2433, 958, 854, 3160, 253, 2781, 2626, 2254, 3876, 3763, 620, 2677, 2415, 343, 2508, 2146, 1103, 4112, 3078, 3293, 2066, 2545, 4103, 607, 2984, 1447, 3109, 1418, 1624, 4006, 1237, 3112, 487, 2566, 2157, 3795, 3652, 4123, 2549, 3922, 1101, 516, 3318, 27, 1633, 3238, 18, 712, 1770, 23, 3628, 4068, 649, 1741, 2002, 4019, 2843, 930, 1886, 3940, 1461, 2892, 1510, 1317, 273, 164, 3313, 1263, 3242, 2517, 2124, 3961, 51, 1889, 3026, 984, 2141, 1194, 3537, 3695, 3712, 2633, 3614, 2469, 2430, 4102, 3963, 2908, 1730, 2844, 2493, 166, 1253, 3040, 1737, 1777, 3106, 2546, 225

In [261]:
print(user[1])

[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [263]:
sample_item = item[2873]
sample_user = user[1]
print(sample_item)
print(sample_user)
print(user_swiped[1][1161])
#print(sigmoid(dot(sample_user, sample_item.T)))

[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
1


In [266]:
restaurant_data_clean.iloc[2873]

price            1
japanese         0
chinese          0
bar              0
western          0
indian           1
mediterranean    0
mexican          0
italian          0
local            1
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 2873, dtype: object

In [264]:
print(sorted(item_item_matrix[2873], reverse = True))

[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998,

In [370]:
# User Cuisine
#print(user[0])
# User Swipe
# Restaurant Cuisne
default_recc = recco[0]
for recc in recco:
    print("restaurant name:", restaurant_name_list[recc])
    print("item-item:", item_item_matrix[default_recc][recc])
    print("user swiped:", user_swiped[1][recc], "item-user-similarity:", item_user[1][recc] ,"user-user-similarity:", user_user[1][recc])
    print("collab_content", collab_content[1][recc])
    for i in range(0, 22):
        print(restaurant_data_clean.columns[i+1],"u:", user[1][i], "r:", restaurant_data_clean.iloc[recc, i+1])

restaurant name: al dia isalamic food
item-item: 0.9999999999999998
user swiped: 0 item-user-similarity: 0.9999999999999998 user-user-similarity: 0.6840066427474977
collab_content 1.6840066427474976
japanese u: 0 r: 0
chinese u: 0 r: 0
bar u: 0 r: 0
western u: 0 r: 0
indian u: 1 r: 1
mediterranean u: 0 r: 0
mexican u: 0 r: 0
italian u: 0 r: 0
local u: 1 r: 1
malaysian u: 0 r: 0
hipster u: 0 r: 0
veg u: 0 r: 0
fast_food u: 0 r: 0
thai u: 0 r: 0
korean u: 0 r: 0
halal u: 0 r: 0
dessert u: 0 r: 0
seafood u: 0 r: 0
health u: 0 r: 0
french u: 0 r: 0
vietnamese u: 0 r: 0
michellin u: 0 r: 0
restaurant name: sri murugan trading
item-item: 0.7071067811865475
user swiped: 0 item-user-similarity: 0.7071067811865475 user-user-similarity: 0.8996272542381021
collab_content 1.6067340354246495
japanese u: 0 r: 0
chinese u: 0 r: 0
bar u: 0 r: 0
western u: 0 r: 0
indian u: 1 r: 1
mediterranean u: 0 r: 0
mexican u: 0 r: 0
italian u: 0 r: 0
local u: 1 r: 0
malaysian u: 0 r: 0
hipster u: 0 r: 0
veg u: 0 r

In [303]:
# User Cuisine
#print(user[0])
# User Swipe
# Restaurant Cuisne
default_recc = recco[0]
for recc in recco:
    print("restaurant name:", restaurant_name_list[recc])
    print("item-item:", item_item_matrix[default_recc][recc])
    print("user swiped:", user_swiped[0][recc], "item-user-similarity:", item_user[0][recc] ,"user-user-similarity:", user_user[0][recc])
    print("collab_content", collab_content[0][recc])
    for i in range(0, 22):
        print(restaurant_data_clean.columns[i+1],"u:", user[0][i], "r:", restaurant_data_clean.iloc[recc, i+1])

restaurant name: food yo cafe
item-item: 1.0
user swiped: 0 item-user-similarity: 1.0 user-user-similarity: 1.0828091677327318
collab_content 2.082809167732732
japanese u: 0 r: 0
chinese u: 0 r: 0
bar u: 0 r: 0
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 0 r: 0
mexican u: 0 r: 0
italian u: 0 r: 0
local u: 1 r: 1
malaysian u: 0 r: 0
hipster u: 0 r: 0
veg u: 0 r: 0
fast_food u: 0 r: 0
thai u: 0 r: 0
korean u: 0 r: 0
halal u: 0 r: 0
dessert u: 0 r: 0
seafood u: 0 r: 0
health u: 0 r: 0
french u: 0 r: 0
vietnamese u: 0 r: 0
michellin u: 0 r: 0
restaurant name: yong bak kut teh
item-item: 0.7071067811865475
user swiped: 0 item-user-similarity: 0.7071067811865475 user-user-similarity: 1.0279320043980489
collab_content 1.7350387855845963
japanese u: 0 r: 0
chinese u: 0 r: 1
bar u: 0 r: 0
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 0 r: 0
mexican u: 0 r: 0
italian u: 0 r: 0
local u: 1 r: 1
malaysian u: 0 r: 0
hipster u: 0 r: 0
veg u: 0 r: 0
fast_food u: 0 r: 0
thai u: 0 r: 0
kor

## Miscallaneous from here on

In [255]:
df_grab = pd.read_csv("complete_grab.csv")

In [43]:
import json
from ast import literal_eval

df_grab

df_tran = pd.DataFrame({})
ind = 0
for cuisine_arr in df_grab['cuisine']:
    i = 1
    dict_foodie = {}
    
    if(type(cuisine_arr) != float):
        cuisine_arr = literal_eval(cuisine_arr)
        for cuisine in cuisine_arr:
            string_iter = 'cuisine'+str(i)
            dict_foodie[string_iter] = 1
            i=i+1
    df_foodie = pd.DataFrame.from_records(dict_foodie, index = [ind])
    df_foodie = pd.concat([df_grab, df_foodie])
    ind+=1

In [21]:
df_foodie

,cuisine1,cuisine10,cuisine11,cuisine12,cuisine13,cuisine14,cuisine15,cuisine16,cuisine17,cuisine18,...,cuisine42,cuisine43,cuisine44,cuisine45,cuisine46,cuisine5,cuisine6,cuisine7,cuisine8,cuisine9
9661,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
